In [ ]:
import tensorflow as tf
import time
import shutil
import os

In [ ]:
#Since we're dealing with image files which are big files, GPU will be used for enhanced performance in terms of computations
#Using Kaggle IDE, GPU is integrated & has been activated
#Checking GPU Activation / Listing number of GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))


In [ ]:
#display working dir
os.listdir('./')

In [ ]:
#this is where the ds is stored
os.listdir('/kaggle/input/')

In [ ]:
#importing important libs
import numpy as np #mathematical fn
import pandas as pd #data manipulation /analysis
import seaborn as sns #statistical data viz
sns.set_style('darkgrid')

import glob #file/path retreival matching pattern
import matplotlib.pyplot as plt #ploting interface


In [ ]:
from tqdm import tqdm #tracking bar

#from matplotlib.pyplot import imshow #image in grayscale

from IPython.core.display import display, HTML

from PIL import Image #To represent Python Imaging Lib Image
from sklearn.metrics import confusion_matrix, classification_report #evaluation stage

In [ ]:
#NOW let's verify class distribution
# NB: It is important to have a balanced / non-biased dataset  - to prevent model frombeing biased to majority class

train_image_names = glob.glob('../input/foodnonfood/TRAIN/TRAIN/*/*.jpg') #READING OF TRAINIiNG imgs from path
print("Total number of training images: ", len(train_image_names))

In [ ]:
# make train_image_names as series object
train_image_names = pd.Series(train_image_names)
train_df = pd.DataFrame()

# generate Filename field
train_df['Filename'] = train_image_names.map(lambda img_name: img_name.split("/")[-1])

# generate ClassId field
train_df['ClassId'] = train_image_names.map(lambda img_name: (img_name.split("/")[-2]))
class_id_distribution = train_df['ClassId'].value_counts()
class_id_distribution

#we can already see that the data is almost balance with food class slightly bigger

In [ ]:
#Now let's visualize the classes in the training; Here we see that the classes are balanced

plt.figure(figsize=(10,5))
plt.bar(class_id_distribution.index, class_id_distribution.values, color=['salmon','cornflowerblue'])
plt.title(label="Training Set Distribution")
plt.xlabel(('Food                                    Vs                                      Non Food'))
plt.show()       


In [ ]:
#TEST dataset
test = glob.glob('../input/foodnonfood/TEST/TEST/*/*.jpg')
print("Total number of test images: ", len(test))

# make train_image_names as serie object
test = pd.Series(test)
testdf = pd.DataFrame()

# generate Filename field
testdf['Filename'] = test.map(lambda img_name: img_name.split("/")[-1])

# generate ClassId field
testdf['ClassId'] = test.map(lambda img_name: (img_name.split("/")[-2]))
class_id_distribution2 = testdf['ClassId'].value_counts()
class_id_distribution2

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(class_id_distribution2.index, class_id_distribution2.values, color=['salmon','cornflowerblue'])
plt.title(label="Test Set Distribution")
plt.xlabel(('Food                                    Vs                                      Non Food'))
plt.show()

Now that the data distribution was checked, we saw that the data does not have high bias. Also, our data will be taken in random batches to ensure biasness is not a problem

In [ ]:

#Lets import some ML Libs

from tensorflow import keras #AI interface for TF Lib
# from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, GlobalAveragePooling2D, MaxPooling2D,BatchNormalization, Flatten #used in DL model layers
from tensorflow.keras.optimizers import Adam, Adamax, RMSprop #to optimize performance
from tensorflow.keras.metrics import categorical_crossentropy #Computes the crossentropy metric between the labels and predictions.
from tensorflow.keras import regularizers # Regularizers for applying penalties on layer parameters during optimization.
from tensorflow.keras.preprocessing.image import ImageDataGenerator #For generating image Batches (with augmentation)
from tensorflow.keras.models import Model, load_model, Sequential #seq to group layer stacks linearly into a Model (tf.keras.Model)
from tensorflow.keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
#as all data need to be pre-processed, here Data augmentation is used to produce favourable results
#Augmentation using ImageData Generator
train='../input/foodnonfood/TRAIN/TRAIN'
train_datagen = ImageDataGenerator(
        rescale=1./255, #to transform every pixel value from range [0,255] -> [0,1]
        shear_range=0.2,#shifting the pixels horizontally
        zoom_range=0.2,
        rotation_range=90,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
#a batch size of 32/64 is an optimum size - smaller batches are better for  stability (Brownlee,2019)
#reading the training images
training_set = train_datagen.flow_from_directory(
                    train,
                    target_size=(224, 224),
                    batch_size=64,
                    class_mode='binary')

In [ ]:
#reading test img
test='../input/foodnonfood/TEST/TEST'
test_datagen=ImageDataGenerator(rescale=1./255)
testset=test_datagen.flow_from_directory(test,target_size=(224,224),batch_size=32,class_mode='binary')

Now that the data has been augemented, let's visualise the images in the DS

In [ ]:
#First let's visualize the Train set
plt.figure(figsize=(15, 15)) #image displayed in size 15X15
for i in range(0, 10):
    plt.subplot(2, 5, i+1) #2 rows with 5 images per row
    for X_batch, Y_batch in training_set:
        image = X_batch[0]        
        dic = {1:'food', 0:'non-food'} 
        plt.title(dic.get(Y_batch[0]))
        plt.axis('off')
        plt.imshow(np.squeeze(image),interpolation='nearest') # nearest will display an image without trying to interpolate between pixels since output display resolution != image resolution
        break
plt.tight_layout()
plt.show()

Images all have different orientation but the image are all of good quality 

In [ ]:
#Let's visualize the testset

plt.figure(figsize=(15, 15)) #image displayed in size 15X15
for i in range(0, 10):
    plt.subplot(2, 5, i+1) #2 rows with 5 images per row
    for X_batch, Y_batch in testset:
        image = X_batch[0]        
        dic = {1:'food', 0:'non-food'} 
        plt.title(dic.get(Y_batch[0]))
        plt.axis('off')
        plt.imshow(np.squeeze(image),interpolation='nearest') # nearest will display an image without trying to interpolate between pixels since output display resolution != image resolution
        break
plt.tight_layout()
plt.show()

In [ ]:
#shape of the sets
print("training set shape is",training_set[-1])
print("test set shape is",testset[-1])

#verifying current image shapes
print("training images shape is",training_set.image_shape)
print("test images shape is",testset.image_shape)

Let's build the model

In [ ]:
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, MaxPooling2D #layers to be used in the CNN arch

In [ ]:
model1= Sequential()
model1.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=[224,224,3]))#images are RGB so last dimension is 3
model1.add(MaxPool2D(pool_size=2,strides=2))
model1.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model1.add(MaxPool2D(pool_size=2,strides=2))
model1.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model1.add(MaxPool2D(pool_size=2,strides=2))
model1.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
model1.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
model1.add(MaxPool2D(pool_size=2,strides=2))
model1.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu'))
model1.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu'))
model1.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu'))
model1.add(MaxPool2D(pool_size=2,strides=2))
#model1.add(Dropout(0.2))
model1.add(Flatten())#Flattening is No brainer and it simply converts a multi-dimensional object to one-dimensional by re-arranging the elements.
model1.add(Dropout(0.2))
#model1.add(Dense(units=1024,activation='relu'))
model1.add(Dense(units=512,activation='relu'))
model1.add(Dense(units=128,activation='relu'))
model1.add(Dense(units=64,activation='relu'))
model1.add(Dense(units=1,activation='sigmoid')) #only one output required

In [ ]:
# Metrics for model
METS = [
      
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]
#compiling model
#binary crossentropy used since this is binary classification model; output has only two classes
model1.compile(optimizer='SGD',loss='binary_crossentropy',metrics=['accuracy',METS])


In [ ]:
model1.summary()

In [ ]:
#model Plotting 
from tensorflow.keras.utils import plot_model
plot_model(model1, show_shapes=True, show_layer_names=True, rankdir='TB', expand_nested=True)

In [ ]:
History = model1.fit(training_set,
                      validation_data=(testset),
                      epochs=25,
                      callbacks=[EarlyStopping(monitor='val_accuracy',patience=10)]) 
#Using early stopping so that if the validation accuracy does not increase 5X conscutively
#model is then stopped; i.e not all 25 epochs will be completed

In [ ]:
#*others used to catch other output being returned
#evaluation of trained model onto the test set
loss, acc, prec, recall, auc, *others = model1.evaluate(testset) 
acc

**Let's Plot the iterations for Model 1**

In [ ]:
#Plot Accuracy
plt.figure()
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model 1 model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#Plot LOSS

plt.figure()
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model 1  loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'Test'], loc='upper left')
plt.show()

Plots end here

Hyper Parameter Tuning


In [ ]:
Tuned= Sequential()#lets make the model more complex to prevent underfit
Tuned.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=[224,224,3],kernel_initializer='he_uniform'))
Tuned.add(MaxPool2D(pool_size=2,strides=2))
Tuned.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
Tuned.add(MaxPool2D(pool_size=2,strides=2))
Tuned.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
Tuned.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
Tuned.add(MaxPool2D(pool_size=2,strides=2))
Tuned.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
Tuned.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
Tuned.add(MaxPool2D(pool_size=2,strides=2))
Tuned.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu'))
Tuned.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu'))
#Tuned.add(MaxPool2D(pool_size=2,strides=2))
Tuned.add(Conv2D(filters=512,kernel_size=(3,3),activation='relu'))
Tuned.add(MaxPool2D(pool_size=2,strides=2))
Tuned.add(Dropout(0.2)) #add dropout to 0.2
Tuned.add(Flatten())
Tuned.add(Dropout(0.2))
Tuned.add(Dense(units=1024,activation='relu'))
Tuned.add(Dense(units=512,activation='relu'))
Tuned.add(Dense(units=128,activation='relu'))
Tuned.add(Dropout(0.2)) 
Tuned.add(Dense(units=64,activation='relu'))
Tuned.add(Dense(units=1,activation='sigmoid')) #only one output required

In [ ]:
#opt = RMSprop(learning_rate= 0.00001, momentum=0.8, epsilon=1e-07)  #changing optimizer & adjust learning rate

Tuned.compile(loss='binary_crossentropy',
              optimizer= 'Adamax',
              metrics=['accuracy',METS])

In [ ]:
Tuned.summary()

In [ ]:
plot_model(Tuned,show_shapes=True, show_layer_names=True, rankdir='TB', expand_nested=True)

In [ ]:
History2 = Tuned.fit(training_set,
                      validation_data=(testset),
                      epochs=100,#increase iterations
                      callbacks=[EarlyStopping(monitor='val_accuracy',patience=10)]) 
#Using early stopping so that if the validation accuracy does not increase 10X conscutively
#model is then stopped; i.e not all 100 epochs will be completed

In [ ]:
loss_tuned, acc_tuned, prec_tune,rec_tuned, auc_tuned,*others = Tuned.evaluate(testset) 
acc_tuned

In [ ]:
#Plot Accuracy
plt.figure()
plt.plot(History2.history['accuracy'])
plt.plot(History2.history['val_accuracy'])
plt.title('Tuned model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Test'], loc='upper left')
plt.xlim([0,50])
plt.ylim([0.0,1.2])
plt.show()

In [ ]:
#Plot LOSS

plt.figure()
plt.plot(History2.history['loss'])
plt.plot(History2.history['val_loss'])
plt.title('Tuned Model  loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'Test'], loc='upper left')
plt.xlim([0,50])
plt.ylim([0.0,1.2])
plt.show()

Compare the models

In [ ]:
#ACCURACY Evaluation

plt.figure()
plt.plot(History.history['accuracy'],label='Model 1 Train',linestyle='dashed')
plt.plot(History.history['val_accuracy'],label='Model 1 Test')
plt.plot(History2.history['accuracy'],label='Tuned  train',linestyle='dashed')
plt.plot(History2.history['val_accuracy'],label='Tuned  Test')
plt.title('Performance: ACCURACY',fontsize=20)
plt.ylabel('Accuracy',fontsize=15)
plt.xlabel('epoch',fontsize=15)
plt.legend( loc='lower right',fontsize=15)
plt.xlim([0,50])
plt.ylim([0.0,1.2])
plt.rcParams['figure.figsize'] = [15, 15]
plt.show()

In [ ]:
#LOSS Evaluation

plt.figure()
plt.plot(History.history['loss'],label=' Model 1 Train',linestyle='dashed')
plt.plot(History.history['val_loss'],label='MB Model Test')
plt.plot(History2.history['loss'],label='Tuned Model train',linestyle='dashed')
plt.plot(History2.history['val_loss'],label='Tuned Model Test')
plt.title('Performance: LOSS',fontsize=20)
plt.ylabel('LOSS',fontsize=15)
plt.xlabel('epoch',fontsize=15)
plt.legend( loc='upper left',fontsize=15)
plt.xlim([0,50])
plt.ylim([0.0,1.00])
plt.rcParams['figure.figsize'] = [8, 8]
plt.show()

Now lets start the predictions

In [ ]:
Prediction=Tuned.predict(testset,verbose=1)

View the distribution of predictions

In [ ]:
import scipy
bins = [0.0,0.2,0.4,0.6,0.8,1.0]
mn, std = scipy.stats.norm.fit(Prediction)

In [ ]:
print("Mean of predictions= ",mn)

In [ ]:
plt.hist(Prediction,[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
plt.title("histogram") 
plt.xlim([0,1])
plt.show()

Lets viz The images and corresponding predictions

In [ ]:
plt.figure(figsize=(20,20))
for i in range(0+0,3+0):
  plt.subplot(1, 3, (i-0)+1)
  if Prediction[i, 0] >= 0.5: #0.5 can be a good cutoff val based on mean of preds
      out = ('{:.2%} probability of being  Food case'.format(Prediction[i][0]))
      
      
  else: 
      out = ('{:.2%} probability of being Non-Food case'.format(1-Prediction[i][0]))
      
      

  plt.title(out+"\n Actual case : "+ dic.get(Y_batch[i]))    
  plt.imshow(np.squeeze(X_batch[i]))
  plt.axis('off')
plt.show()

In [ ]:
testset.reset()
x=np.concatenate([testset.next()[0] for i in range(testset.__len__())])
y=np.concatenate([testset.next()[1] for i in range(testset.__len__())])
print(x.shape)
print(y.shape)

In [ ]:
preds = Tuned.predict(testset,verbose=1)

In [ ]:
predictions = preds.copy()
predictions[predictions <= 0.5] = 0
predictions[predictions > 0.5] = 1

In [ ]:
print(classification_report(y_true=y,y_pred=predictions,target_names =['NONFOOD','FOOD']))

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
cm = pd.DataFrame(data=confusion_matrix(y, predictions, labels=[0, 1]),index=["Actual Non Food", "Actual Food"],
columns=["Predicted Non Food", "Predicted Food"])
import seaborn as sns
sns.heatmap(cm,annot=True,fmt="d")

In [ ]:
Tuned.summary()


In [ ]:
last_layer='conv2d_50'

# GRAD CAM

In [ ]:
from tensorflow.keras.models import Model
import cv2


def GradCam(model, imarr, layer_name, eps=1e-8): #epsil value def
    

    gMod = Model(
			inputs=[model.inputs],
			outputs=[model.get_layer(layer_name).output,
				model.output])
    
    with tf.GradientTape() as tape:
      inputs = tf.cast(imarr, tf.float32)
      (convOutputs, predictions) = gMod(inputs)
      loss = predictions[:, 0]
    gradientss = tape.gradient(loss, convOutputs)
    
    csCNVOut = tf.cast(convOutputs > 0, "float32")
    csgradie = tf.cast(gradientss > 0, "float32")
    guidedgradientss = csCNVOut * csgradie * gradientss
    convOutputs = convOutputs[0]
    guidedgradientss = guidedgradientss[0]
    wgts = tf.reduce_mean(guidedgradientss, axis=(0, 1))
    gdcam = tf.reduce_sum(tf.multiply(wgts, convOutputs), axis=-1)
    (w, h) = (imarr.shape[2], imarr.shape[1])
    hm = cv2.resize(gdcam.numpy(), (w, h))
    numer = hm - np.min(hm)
    denom = (hm.max() - hm.min()) + eps
    hm = numer / denom
    return hm


def sigmoid(x, a, b, c):
    return c / (1 + np.exp(-a * (x-b)))

def supimp(img_bgr, gdcam, thresh, emphasize=False):
    hm = cv2.resize(gdcam, (img_bgr.shape[1], img_bgr.shape[0]))
    if emphasize:
        hm = sigmoid(hm, 50, thresh, 1)
    hm = np.uint8(255 * hm)
    hm = cv2.applyColorMap(hm, cv2.COLORMAP_JET)
    
    hif = .8
    supimpd_img = hm * hif + img_bgr * hif
    supimpd_img = np.minimum(supimpd_img, 255.0).astype(np.uint8)  # scale 0 to 255  
    supimpd_img_rgb = cv2.cvtColor(supimpd_img, cv2.COLOR_BGR2RGB)
    #supimpd_img_rgb = supimpd_img
    return supimpd_img_rgb

In [ ]:
j=5

In [ ]:
MR=X_batch[j]

In [ ]:
plt.imshow(MR)

In [ ]:
MR.shape

In [ ]:
MRre = cv2.resize(MR,(224,224))     # resize image to match model's expected sizing
MRre = np.reshape(MRre,[1,224,224,3]) 

In [ ]:
MRre.shape

In [ ]:
predtionNF=Tuned.predict(MRre)

In [ ]:
predtionNF

In [ ]:
if (predtionNF>=0.5):
    print('Predicted Class is FOOD')
    print('Actual Class'+dic.get(Y_batch[j]))
    grad_cam3=GradCam(Tuned,MRre,last_layer)
    plt.imsave('./noods.png', MR)
    img_path='./noods.png'
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224),3)
    grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)
    #grad_cam_supimpd3 = supimp(MR, grad_cam3, 0.5, emphasize=False)

    plt.figure(figsize=(15, 15))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(MR)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam3)
    #plt.axis('off')
    plt.title('Grad cam built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()
else:
    print('Predicted Class is Non-FOOD')
    print('Actual Class',dic.get(Y_batch[j]))
    grad_cam3=GradCam(Tuned,MRre,last_layer)
    grad_cam_supimpd3 = supimp(MR, grad_cam3, 0.5, emphasize=False)

    plt.figure(figsize=(15, 15))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(MR)
    #plt.axis('off')
    plt.title('Original Image',)
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam3)
    #plt.axis('off')
    plt.title('Grad cam built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()

In [ ]:
k=3
MR2=X_batch[k]
plt.imshow(MR2)

In [ ]:
MRre2 = cv2.resize(MR2,(224,224))     # resize image to match model's expected sizing
MRre2 = np.reshape(MRre2,[1,224,224,3]) 
predtionNF2=Tuned.predict(MRre2)

In [ ]:
predtionNF2

In [ ]:
if (predtionNF2>=0.5):
    print('Predicted Class is FOOD')
    print('Actual Class is ', dic.get(Y_batch[k]))
    grad_cam443=GradCam(Tuned,MRre2,last_layer)
    plt.imsave('./came.png', MR2)
    img_path='./came.png'
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224),3)
    grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)
    #grad_cam_supimpd3 = supimp(MR2, grad_cam443, 0.1, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(MR2)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()
else:
    print('Predicted Class is NON FOOD')
    print('Actual Class is ', dic.get(Y_batch[k]))
    grad_cam443=GradCam(Tuned,MRre2,last_layer)
    plt.imsave('./came.png', MR2)
    img_path='./came.png'
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224),3)
    grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)
    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(MR2)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()

In [ ]:
k=15

In [ ]:
MR2=X_batch[k]
plt.imshow(MR2)

In [ ]:
MRre2 = cv2.resize(MR2,(224,224))     # resize image to match model's expected sizing
MRre2 = np.reshape(MRre2,[1,224,224,3]) 
predtionNF2=Tuned.predict(MRre2)

In [ ]:
predtionNF2

In [ ]:
if (predtionNF2>=0.5):
    print('Predicted Class is FOOD')
    print('Actual Class is ', dic.get(Y_batch[k]))
    grad_cam443=GradCam(Tuned,MRre2,last_layer)
    
    grad_cam_supimpd3 = supimp(MR2, grad_cam443, 1.5, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(MR2)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()
else:
    print('Predicted Class is NON FOOD')
    print('Actual Class is ', dic.get(Y_batch[k]))
    grad_cam443=GradCam(Tuned,MRre2,last_layer)
    plt.imsave('./cheval.png', MR2)
    img_path='./cheval.png'
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224),3)
    grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)
    #grad_cam_supimpd3 = supimp(MR2, grad_cam443, 1.5, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(MR2)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()

In [ ]:
#Let's visualize the testset

plt.figure(figsize=(15, 15)) #image displayed in size 15X15
for i in range(0, 10):
    plt.subplot(2, 5, i+1) #2 rows with 5 images per row
    for x, y in testset:
        image = x[0]        
        dic = {1:'food', 0:'non-food'} 
        plt.title(dic.get(y[0]))
        plt.axis('off')
        plt.imshow(np.squeeze(image),interpolation='nearest') # nearest will display an image without trying to interpolate between pixels since output display resolution != image resolution
        break
plt.tight_layout()
plt.show()

In [ ]:
FD=11
plt.imshow(x[FD])
plt.title(dic.get(y[FD]))

In [ ]:
img_array3=x[FD]
img2 = cv2.resize(img_array3,(224,224))     # resize image to match model's expected sizing
img2 = np.reshape(img_array3,[1,224,224,3]) 

In [ ]:
preFD=Tuned.predict(img2)

In [ ]:
preFD

In [ ]:
if (preFD>=0.5):
    print('Predicted Class is FOOD')
    print('{:.2%} probability of being  Food case'.format(preFD[0][0]))
    print('Actual Class is ', dic.get(y[FD]))
    grad_cam443=GradCam(Tuned,img2,last_layer)
    grad_cam_supimpd3 = supimp(img_array3, grad_cam443, 0.1, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img_array3)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()
else:
    print('Predicted Class is NON FOOD')
    print('{:.2%} probability of being NON Food case'.format(1-preFD[0][0]))
    print('Actual Class is ', dic.get(y[FD]))
    grad_cam443=GradCam(Tuned,img2,last_layer)
    grad_cam_supimpd3 = supimp(img_array3, grad_cam443, 0.1, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img_array3)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()

In [ ]:
NFD=0
plt.imshow(X_batch[NFD])
plt.title(dic.get(Y_batch[NFD]))

In [ ]:
img_array4=X_batch[NFD]
img3 = cv2.resize(img_array4,(224,224))     # resize image to match model's expected sizing
img3 = np.reshape(img_array4,[1,224,224,3]) 

In [ ]:
preNFD=Tuned.predict(img3)

In [ ]:
preNFD

In [ ]:
if (preNFD>=0.5):
    print('Predicted Class is FOOD')
    print('{:.2%} probability of being  Food case'.format(preNFD[0][0]))
    print('Actual Class is ', dic.get(Y_batch[NFD]))
    grad_cam443=GradCam(Tuned,img3,last_layer)
    plt.imsave('./Painchoc.png', img_array4)
    img_path='./Painchoc.png'
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224),3)
    grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)
    #grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img_array4)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()
else:
    print('Predicted Class is NON FOOD')
    print('{:.2%} probability of being NON Food case'.format(1-preNFD[0][0]))
    print('Actual Class is ', dic.get(y[NFD]))
    grad_cam443=GradCam(Tuned,img3,last_layer)
    grad_cam_supimpd3 = supimp(img_array4, grad_cam443, 0.1, emphasize=False)

    plt.figure(figsize=(10, 10))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img_array4)
    plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()

In [ ]:
plt.imshow(img_array4)

In [ ]:
#model_builder = keras.applications.xception.Xception
img_size = (224, 224)
preprocess_input = keras.applications.xception.preprocess_input
#decode_predictions = keras.applications.xception.decode_predictions
#img_array = preprocess_input(get_img_array(img_path, size=img_size))


In [ ]:
last_layer

In [ ]:
grad_cam443=GradCam(Tuned,img3,last_layer)
#grad_cam_supimpd3 = supimp(img_array, grad_cam443, 1.0, emphasize=False)


In [ ]:
grad_cam443.shape

In [ ]:
preFD=Tuned.predict(img2)

In [ ]:
img_array = preprocess_input(get_img_array(img_path, size=img_size))

In [ ]:
if (preFD>=0.5):
    print('Predicted Class is FOOD')
    print('{:.2%} probability of being  Food case'.format(preFD[0][0]))
    print('Actual Class is ', dic.get(y[FD]))
    grad_cam443=GradCam(Tuned,img2,last_layer)
    grad_cam_supimpd3 = supimp(img_array3, grad_cam443, 0.1, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img_array3)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()
else:
    print('Predicted Class is NON FOOD')
    print('{:.2%} probability of being NON Food case'.format(1-preFD[0][0]))
    print('Actual Class is ', dic.get(y[FD]))
    grad_cam443=GradCam(Tuned,img2,last_layer)
    plt.imsave('./Guitare.png', img_array3)
    img_path='./Guitare.png'
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224),3)
    grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img_array3)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()

In [ ]:
#img_array = preprocess_input(get_img_array(img_path, size=img_size))

In [ ]:
#img_array = preprocess_input(get_img_array(img_path, size=img_size))

In [ ]:
k=30
MR2=X_batch[k]
plt.imshow(MR2)

MRre2 = cv2.resize(MR2,(224,224))     # resize image to match model's expected sizing
MRre2 = np.reshape(MRre2,[1,224,224,3]) 
predtionNF2=Tuned.predict(MRre2)

In [ ]:
if (predtionNF2>=0.5):
    print('Predicted Class is FOOD')
    print('Actual Class is ', dic.get(Y_batch[k]))
    grad_cam443=GradCam(Tuned,MRre2,last_layer)
    grad_cam_supimpd3 = supimp(MR2, grad_cam443, 0.1, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(MR2)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()
else:
    print('Predicted Class is NON FOOD')
    print('Actual Class is ', dic.get(Y_batch[k]))
    grad_cam443=GradCam(Tuned,MRre2,last_layer)
    plt.imsave('./Ciel.png', MR2)
    img_path='./Ciel.png'
    #display(Image.open(img_path))
    #grad_cam443=GradCam(Tuned,MRre2,last_layer)
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224),3)
    grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)
    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(MR2)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()

In [ ]:
NFD=31
plt.imshow(X_batch[NFD])
plt.title(dic.get(Y_batch[NFD]))



In [ ]:
img_array4=X_batch[NFD]
img3 = cv2.resize(img_array4,(224,224))     # resize image to match model's expected sizing
img3 = np.reshape(img_array4,[1,224,224,3]) 

In [ ]:
preNFD=Tuned.predict(img3)

In [ ]:
if (preNFD>=0.5):
    print('Predicted Class is FOOD')
    print('{:.2%} probability of being  Food case'.format(preNFD[0][0]))
    print('Actual Class is ', dic.get(Y_batch[NFD]))
    grad_cam443=GradCam(Tuned,img3,last_layer)
    plt.imsave('./macaroni.png', img_array4)
    img_path='./macaroni.png'
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224),3)
    grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)
    #grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img_array4)
    #plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    #plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    #plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()
else:
    print('Predicted Class is NON FOOD')
    print('{:.2%} probability of being NON Food case'.format(1-preNFD[0][0]))
    print('Actual Class is ', dic.get(y[NFD]))
    grad_cam443=GradCam(Tuned,img3,last_layer)
    grad_cam_supimpd3 = supimp(img_array4, grad_cam443, 0.1, emphasize=False)

    plt.figure(figsize=(10, 10))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img_array4)
    plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()

In [ ]:
NFD=23
plt.imshow(X_batch[NFD])
plt.title(dic.get(Y_batch[NFD]))


In [ ]:
img_array4=X_batch[NFD]
img3 = cv2.resize(img_array4,(224,224))     # resize image to match model's expected sizing
img3 = np.reshape(img_array4,[1,224,224,3]) 

In [ ]:
preNFD=Tuned.predict(img3)

In [ ]:
if (preNFD>=0.5):
    print('Predicted Class is FOOD')
    print('{:.2%} probability of being  Food case'.format(preNFD[0][0]))
    print('Actual Class is ', dic.get(Y_batch[NFD]))
    grad_cam443=GradCam(Tuned,img3,last_layer)
    plt.imsave('./cafe.png', img_array4)
    img_path=('./cafe.png')
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224),3)
    grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)
    #grad_cam_supimpd3 = supimp(img, grad_cam443, 1.0, emphasize=False)

    plt.figure(figsize=(20, 20))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img_array4)
    plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()
else:
    print('Predicted Class is NON FOOD')
    print('{:.2%} probability of being NON Food case'.format(1-preNFD[0][0]))
    print('Actual Class is ', dic.get(y[NFD]))
    grad_cam443=GradCam(Tuned,img3,last_layer)
    grad_cam_supimpd3 = supimp(img_array4, grad_cam443, 0.1, emphasize=False)

    plt.figure(figsize=(10, 10))
    ax = plt.subplot(1, 3, 1)
    plt.imshow(img_array4)
    plt.axis('off')
    plt.title('Original Image')
    ax = plt.subplot(1, 3, 2)
    plt.imshow(grad_cam443)
    plt.axis('off')
    plt.title('Grad Cam Image built')
    ax = plt.subplot(1, 3, 3)
    plt.imshow(grad_cam_supimpd3)
    plt.axis('off')
    plt.title('Last CONV Grad-CAM heat-map')
    plt.tight_layout()

In [ ]:
print("END")